In [ ]:
# Gerekli kütüphaneleri yükle
!pip install anthropic openai yfinance pandas numpy requests -q

print("✅ Kurulum tamamlandı!")

In [ ]:
from google.colab import userdata
import os

try:
    ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')
    print("✅ Anthropic API anahtarı yüklendi")
except:
    print("❌ ANTHROPIC_API_KEY bulunamadı! Lütfen Colab Secrets'a ekleyin.")
    ANTHROPIC_API_KEY = None

try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    print("✅ OpenAI API anahtarı yüklendi")
except:
    print("⚠️ OPENAI_API_KEY bulunamadı (opsiyonel)")
    OPENAI_API_KEY = None

# Hangisini kullanmak istediğini seç
USE_ANTHROPIC = True  # False yaparsan OpenAI kullanır

if USE_ANTHROPIC and not ANTHROPIC_API_KEY:
    raise Exception("Anthropic kullanmak için ANTHROPIC_API_KEY gerekli!")
elif not USE_ANTHROPIC and not OPENAI_API_KEY:
    raise Exception("OpenAI kullanmak için OPENAI_API_KEY gerekli!")

print(f"\n🤖 Kullanılacak API: {'Anthropic Claude' if USE_ANTHROPIC else 'OpenAI GPT'}")

In [ ]:
import anthropic
import openai

class LLMClient:
    """Anthropic veya OpenAI'yi kullanan birleşik client"""
    
    def __init__(self, use_anthropic=True):
        self.use_anthropic = use_anthropic
        
        if use_anthropic:
            self.client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)
            self.model = "claude-sonnet-4-20250514"  # En yeni model
        else:
            openai.api_key = OPENAI_API_KEY
            self.model = "gpt-4"
    
    def chat(self, messages, temperature=0, stop=None):
        """Birleşik chat fonksiyonu"""
        
        if self.use_anthropic:
            # Anthropic için system mesajını ayır
            system_msg = ""
            user_msgs = []
            
            for msg in messages:
                if msg["role"] == "system":
                    system_msg = msg["content"]
                else:
                    user_msgs.append({
                        "role": msg["role"],
                        "content": msg["content"]
                    })
            
            response = self.client.messages.create(
                model=self.model,
                max_tokens=2000,
                temperature=temperature,
                system=system_msg,
                messages=user_msgs,
                stop_sequences=stop or ["PAUSE", "Observation:"]
            )
            return response.content[0].text
        
        else:
            # OpenAI
            response = openai.ChatCompletion.create(
                model=self.model,
                messages=messages,
                temperature=temperature,
                stop=stop or ["PAUSE", "Observation:"]
            )
            return response.choices[0].message.content

# Client'i oluştur
llm_client = LLMClient(use_anthropic=USE_ANTHROPIC)
print(f"✅ LLM Client hazır: {llm_client.model}")

In [ ]:
import yfinance as yf
import requests
from datetime import datetime

# ========================
# FİNANS ARAÇLARI
# ========================

def get_stock_price(ticker: str) -> str:
    """
    Hisse senedi veya kripto için güncel fiyat bilgisi getirir.
    Örnek: get_stock_price: AAPL veya BTC-USD
    """
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        current_price = info.get('currentPrice') or info.get('regularMarketPrice')
        
        if current_price:
            return f"{ticker} güncel fiyat: ${current_price:.2f}"
        else:
            return f"{ticker} için fiyat bilgisi bulunamadı"
    except Exception as e:
        return f"Hata: {e}"


def get_stock_info(ticker: str) -> str:
    """
    Şirket hakkında temel bilgiler getirir (sektör, çalışan sayısı, piyasa değeri vb.)
    Örnek: get_stock_info: MSFT
    """
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        
        result = f"""
📊 {info.get('longName', ticker)} Bilgileri:
- Sektör: {info.get('sector', 'N/A')}
- Endüstri: {info.get('industry', 'N/A')}
- Piyasa Değeri: ${info.get('marketCap', 0):,.0f}
- Çalışan Sayısı: {info.get('fullTimeEmployees', 'N/A'):,}
- Şirket Özeti: {info.get('longBusinessSummary', 'N/A')[:200]}...
        """
        return result.strip()
    except Exception as e:
        return f"Hata: {e}"


def get_financial_ratios(ticker: str) -> str:
    """
    Finansal oranları getirir (P/E, PEG, Debt/Equity vb.)
    Örnek: get_financial_ratios: TSLA
    """
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        
        result = f"""
📈 {ticker} Finansal Oranlar:
- P/E Ratio: {info.get('trailingPE', 'N/A')}
- PEG Ratio: {info.get('pegRatio', 'N/A')}
- Debt/Equity: {info.get('debtToEquity', 'N/A')}
- ROE: {info.get('returnOnEquity', 'N/A')}
- ROA: {info.get('returnOnAssets', 'N/A')}
- Profit Margin: {info.get('profitMargins', 'N/A')}
        """
        return result.strip()
    except Exception as e:
        return f"Hata: {e}"


def get_currency_rate(from_currency: str, to_currency: str = "TRY") -> str:
    """
    Döviz kuru bilgisi getirir. TCMB veya Yahoo Finance üzerinden.
    Örnek: get_currency_rate: USD (TRY'ye çevirir)
    """
    try:
        # Yahoo Finance ticker formatı: USDTRY=X
        ticker = f"{from_currency}{to_currency}=X"
        forex = yf.Ticker(ticker)
        rate = forex.info.get('regularMarketPrice')
        
        if rate:
            return f"1 {from_currency} = {rate:.4f} {to_currency}"
        else:
            return f"{from_currency}/{to_currency} kuru bulunamadı"
    except Exception as e:
        return f"Hata: {e}"


def calculator(expression: str) -> str:
    """
    Matematiksel hesaplama yapar.
    Örnek: calculator: (100 * 1.5) + 200
    """
    try:
        # Güvenlik için sadece sayı ve operatör karakterlerine izin ver
        allowed = set("0123456789+-*/(). ")
        if not all(c in allowed for c in expression):
            return "Hata: Geçersiz karakter"
        
        result = eval(expression)
        return f"{expression} = {result}"
    except Exception as e:
        return f"Hata: {e}"


def get_crypto_price(symbol: str) -> str:
    """
    Kripto para güncel fiyatını getirir.
    Örnek: get_crypto_price: BTC (Bitcoin için)
    """
    try:
        # Yahoo Finance kripto ticker formatı: BTC-USD
        ticker = f"{symbol}-USD"
        crypto = yf.Ticker(ticker)
        price = crypto.info.get('regularMarketPrice')
        
        if price:
            return f"{symbol} güncel fiyat: ${price:,.2f}"
        else:
            return f"{symbol} kripto fiyatı bulunamadı"
    except Exception as e:
        return f"Hata: {e}"


# ARAÇ LİSTESİ
tools = [
    get_stock_price,
    get_stock_info, 
    get_financial_ratios,
    get_currency_rate,
    calculator,
    get_crypto_price
]

# Tool açıklamalarını birleştir
tools_description = "\n\n".join([
    f"{tool.__name__}: {tool.__doc__}" for tool in tools
])

# Tool dictionary (fonksiyon adı -> fonksiyon)
known_actions = {tool.__name__: tool for tool in tools}

print("✅ Finans araçları hazır!")
print(f"📌 Toplam {len(tools)} araç yüklendi")

In [ ]:
import re
import math
from collections import Counter

class FinancialRAG:
    """Finansal dokümanlar için basit RAG motoru"""
    
    def __init__(self):
        self.vector_store = []
        self.document_count = 0
    
    def get_embedding(self, text):
        """Basit kelime frekansı embedding (TF-IDF benzeri)"""
        words = re.findall(r'\w+', text.lower())
        return Counter(words)
    
    def cosine_similarity(self, vec1, vec2):
        """İki vektör arasındaki cosine similarity hesapla"""
        intersection = set(vec1.keys()) & set(vec2.keys())
        
        if not intersection:
            return 0.0
        
        numerator = sum([vec1[x] * vec2[x] for x in intersection])
        sum1 = sum([vec1[x]**2 for x in vec1.keys()])
        sum2 = sum([vec2[x]**2 for x in vec2.keys()])
        
        denominator = math.sqrt(sum1) * math.sqrt(sum2)
        
        return float(numerator) / denominator if denominator else 0.0
    
    def ingest(self, text, doc_name=""):
        """Metni chunk'lara böl ve vektör deposuna ekle"""
        # Cümlelere böl
        sentences = [s.strip() for s in re.split(r'[.!?]+', text) if s.strip()]
        
        # Her 2-3 cümleyi bir chunk yap (daha iyi context için)
        chunks = []
        for i in range(0, len(sentences), 2):
            chunk = ". ".join(sentences[i:i+2]) + "."
            chunks.append(chunk)
        
        # Vector store'a ekle
        for chunk in chunks:
            self.vector_store.append({
                "content": chunk,
                "vector": self.get_embedding(chunk),
                "doc_name": doc_name or f"doc_{self.document_count}"
            })
        
        self.document_count += 1
        print(f"✅ {len(chunks)} chunk eklendi (Toplam: {len(self.vector_store)} chunk)")
    
    def retrieve(self, query, top_k=3):
        """Query'ye en yakın chunk'ları getir"""
        if not self.vector_store:
            return "Veritabanında bilgi yok."
        
        query_vec = self.get_embedding(query)
        scores = []
        
        for item in self.vector_store:
            score = self.cosine_similarity(query_vec, item["vector"])
            scores.append((score, item["content"], item["doc_name"]))
        
        # Score'a göre sırala
        scores.sort(key=lambda x: x[0], reverse=True)
        
        # En iyi top_k sonucu döndür
        results = []
        for score, content, doc_name in scores[:top_k]:
            if score > 0:  # Sadece benzerliği olanları
                results.append(f"[{doc_name}] {content}")
        
        return "\n\n".join(results) if results else "İlgili bilgi bulunamadı."


# RAG Engine'i oluştur
rag_engine = FinancialRAG()

# ========================
# FİNANSAL BİLGİ BANKASI
# ========================

# Finansal Terimler Sözlüğü
financial_terms = """
P/E Ratio (Fiyat/Kazanç Oranı): Hisse senedi fiyatının şirketin hisse başına kazancına bölünmesiyle bulunur. 
Düşük P/E genellikle hisse senedinin ucuz olduğunu, yüksek P/E ise pahalı veya büyüme beklentisi olduğunu gösterir.
Sektör ortalaması 15-20 arasındadır.

PEG Ratio: P/E oranının kazanç büyüme hızına bölünmesiyle bulunur. 
1'den küçük değerler genellikle hissenin değerli olduğunu gösterir.
PEG < 1 değerli, PEG > 2 pahalı kabul edilir.

Debt/Equity (Borç/Özsermaye): Şirketin toplam borcunun özsermayesine oranıdır.
Yüksek oran (>2) finansal risk demektir. Düşük oran (<0.5) güvenli kabul edilir.

ROE (Return on Equity): Özsermaye karlılığı. %15'in üzeri iyi kabul edilir.
Şirketin özsermayesini ne kadar verimli kullandığını gösterir.

Profit Margin (Kar Marjı): Net karın gelire oranı. %10'un üzeri sağlıklı kabul edilir.
Yüksek kar marjı rekabet avantajı gösterir.

Market Cap (Piyasa Değeri): Şirketin toplam değeri.
Large-cap: >10B$, Mid-cap: 2-10B$, Small-cap: <2B$.

Beta: Hisse senedinin piyasaya göre volatilitesi. Beta>1 volatil, Beta<1 stabil demektir.
"""

# Yatırım Stratejileri
investment_strategies = """
Değer Yatırımı (Value Investing): Düşük P/E ve P/B oranlarına sahip, piyasa tarafından değeri anlaşılmamış 
şirketlere yatırım yapmaktır. Warren Buffett'ın kullandığı stratejidir.

Büyüme Yatırımı (Growth Investing): Yüksek büyüme potansiyeli olan, geliri hızla artan şirketlere yatırım.
Genellikle yüksek P/E oranlarına sahiptirler. Teknoloji şirketleri örnek verilebilir.

Momentum Yatırımı: Fiyatı yükselişte olan hisselere yatırım. Teknik analiz kullanılır.
Kısa vadeli işlemler için uygundur.

Temettü Yatırımı (Dividend Investing): Düzenli temettü ödeyen, olgun şirketlere yatırım.
Pasif gelir elde etmek isteyenler için uygundur. Temettü verimi %3'ün üzeri iyi kabul edilir.

Portföy Çeşitlendirmesi: Riski dağıtmak için farklı sektör ve varlık sınıflarına yatırım.
%60 hisse, %30 tahvil, %10 emtia dengeli bir portföy örneğidir.
"""

# Risk Yönetimi
risk_management = """
Stop-Loss: Zararı sınırlamak için belirlenen fiyat seviyesi. Genellikle %5-10 altında konur.
Disiplinli yatırımcılar mutlaka stop-loss kullanmalıdır.

Position Sizing (Pozisyon Büyüklüğü): Tek bir hisseye portföyün maksimum %5-10'unu yatırmak.
Aşırı konsantrasyon riskten korunmak için önemlidir.

Risk/Reward Oranı: Kazanç potansiyelinin risk miktarına oranı. Minimum 1:2 olmalıdır.
Örneğin %10 risk alıyorsanız, %20 kazanç hedeflenmelidir.

Volatilite Analizi: Beta katsayısı ve ATR (Average True Range) ile ölçülür.
Yüksek volatilite yüksek risk demektir. Muhafazakar yatırımcılar düşük beta'lı hisseler tercih etmelidir.

Korelasyon: Varlıklar arasındaki ilişki. Düşük korelasyonlu varlıklar portföy riskini azaltır.
Altın ve hisse senetleri genellikle negatif korelasyonludur.
"""

# Piyasa Göstergeleri
market_indicators = """
RSI (Relative Strength Index): 0-100 arası değer alır. 70 üzeri aşırı alım, 30 altı aşırı satım bölgesidir.
RSI 30'un altındaysa alım fırsatı olabilir.

MACD (Moving Average Convergence Divergence): Trend takibi için kullanılır.
MACD çizgisi sinyal çizgisini yukarı keserse alım sinyalidir.

Moving Averages: 50 günlük ve 200 günlük hareketli ortalamalar önemlidir.
Golden Cross (50 MA > 200 MA) yükseliş, Death Cross ise düşüş sinyalidir.

Volume (İşlem Hacmi): Fiyat artışıyla birlikte artan hacim güçlü trend gösterir.
Düşük hacimli hareketler güvenilir değildir.

VIX (Volatility Index): Korku endeksi olarak bilinir. 20 üzeri yüksek volatilite demektir.
VIX yüksekken piyasa düşüş eğilimindedir.
"""

# Sektör Analizleri
sector_analysis = """
Teknoloji Sektörü: Yüksek büyüme potansiyeli ama volatilite de yüksektir.
P/E oranları genellikle 25-40 arası. Cloud, AI, cybersecurity alt sektörleri öne çıkıyor.

Finans Sektörü: Faiz oranlarından doğrudan etkilenir. Düşük P/E oranlarına sahip.
Banka ve sigorta şirketleri temettü yatırımı için uygundur.

Sağlık Sektörü: Savunmacı sektör, ekonomik daralmalarda daha dirençli.
İlaç şirketleri patent korumalarından faydalanır. Düzenli temettü öderler.

Enerji Sektörü: Petrol fiyatlarına bağlı, yüksek volatilite. 
Yenilenebilir enerji şirketleri geleceğin yıldızları olabilir.

Tüketim Malları: Savunmacı sektör. Coca-Cola, P&G gibi şirketler istikrarlı temettü öder.
Ekonomik krizlerde güvenli liman olarak görülür.
"""

# Bilgi bankasına ekle
print("📥 Finansal bilgi bankası oluşturuluyor...\n")
rag_engine.ingest(financial_terms, "Finansal Terimler")
rag_engine.ingest(investment_strategies, "Yatırım Stratejileri")
rag_engine.ingest(risk_management, "Risk Yönetimi")
rag_engine.ingest(market_indicators, "Piyasa Göstergeleri")
rag_engine.ingest(sector_analysis, "Sektör Analizleri")

print(f"\n✅ RAG bilgi bankası hazır! Toplam {len(rag_engine.vector_store)} bilgi parçası yüklendi.")

In [ ]:
def search_financial_knowledge(query: str) -> str:
    """
    Finansal kavramlar, yatırım stratejileri ve risk yönetimi hakkında bilgi arar.
    Örnek: search_financial_knowledge: P/E ratio nedir
    """
    print(f"\n🔍 [RAG TARAMASI] Bilgi bankasından aranıyor: '{query}'")
    result = rag_engine.retrieve(query, top_k=3)
    
    if "İlgili bilgi bulunamadı" in result:
        return "Bilgi bankasında bu konuyla ilgili bilgi bulunamadı."
    
    return f"📚 Bilgi Bankası Sonuçları:\n{result}"


# RAG tool'unu araç listesine ekle
tools.append(search_financial_knowledge)
known_actions[search_financial_knowledge.__name__] = search_financial_knowledge

# Tool açıklamalarını güncelle
tools_description = "\n\n".join([
    f"{tool.__name__}: {tool.__doc__}" for tool in tools
])

print("✅ RAG tool eklendi!")
print(f"📌 Toplam {len(tools)} araç mevcut")

In [ ]:
import re

class FinancialAgent:
    """ReAct mantığıyla çalışan Finansal Asistan"""
    
    def __init__(self, system_prompt, llm_client):
        self.system_prompt = system_prompt
        self.llm_client = llm_client
        self.messages = []
        
        # System prompt'u ekle
        if self.system_prompt:
            self.messages.append({
                "role": "system",
                "content": system_prompt
            })
    
    def __call__(self, message):
        """Agent'a yeni mesaj gönder"""
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
    
    def execute(self):
        """LLM'den yanıt al"""
        return self.llm_client.chat(
            messages=self.messages,
            temperature=0,
            stop=None
        )


# SYSTEM PROMPT TASARIMI
system_prompt = f"""
Sen bir Finansal Yatırım Danışmanı AI'sın. Türkçe konuşuyorsun.

KRİTİK KURALLAR:
1. MUTLAKA bu formatı kullan:

Thought: [Ne yapacağını düşün]
Action: [araç_adı]: [parametre]
PAUSE

Observation'dan sonra:

Thought: [Yeni düşünce]
Action: [yeni_araç]: [parametre]
PAUSE

veya

Thought: [Final düşünce]
Answer: [Kesin cevabın]

2. "PAUSE" kelimesinden sonra MUTLAKA DUR, başka bir şey yazma
3. Cevabını verirken MUTLAKA "Answer:" ile başla
4. Kafandan hesap yapma, calculator kullan
5. Action'dan sonra Observation bekle, devam etme

MEVCUT ARAÇLAR:
{tools_description}

ÖRNEK 1 (Basit):
Soru: Bitcoin fiyatı nedir?
Thought: Bitcoin'in USD fiyatını öğrenmeliyim
Action: get_crypto_price: BTC
PAUSE

Observation: BTC güncel fiyat: $45,230.50
Thought: Artık cevap verebilirim
Answer: Bitcoin'in güncel fiyatı $45,230.50

ÖRNEK 2 (Karmaşık):
Soru: Apple hissesi alınır mı?
Thought: Önce Apple'ın finansal oranlarına bakmalıyım
Action: get_financial_ratios: AAPL
PAUSE

Observation: P/E Ratio: 28.5...
Thought: P/E oranının ne anlama geldiğini öğrenmeliyim
Action: search_financial_knowledge: P/E ratio yüksek ne demek
PAUSE

Observation: P/E 20'nin üzeri pahalı kabul edilir...
Thought: Şimdi sektör bilgisine de bakalım
Action: get_stock_info: AAPL
PAUSE

Observation: Sektör: Technology...
Thought: Artık analiz yapabilirim
Answer: Apple hisse senedi analizi:
- P/E: 28.5 (sektör ortalamasının üzerinde, pahalı)
- Güçlü şirket ama fiyat yüksek
- Düzeltme beklenebilir
⚠️ Yatırım tavsiyesi değildir

ÖNEMLİ: 
- Her Action sonrası sadece "PAUSE" yaz, başka hiçbir şey yazma
- Cevap verirken "Answer:" kullan
- Türkçe yaz
""".strip()

print("✅ Agent system prompt hazır!")
print(f"📏 Prompt uzunluğu: {len(system_prompt)} karakter")

In [ ]:
action_pattern = re.compile(r'^Action:\s*(\w+):\s*(.*)$', re.IGNORECASE | re.MULTILINE)

def run_agent(question, max_turns=10, verbose=True):
    """
    ReAct döngüsünü çalıştır - İyileştirilmiş versiyon
    """
    
    # Agent'ı oluştur
    agent = FinancialAgent(system_prompt, llm_client)
    
    turn = 0
    next_prompt = question
    
    if verbose:
        print("\n" + "="*60)
        print(f"❓ SORU: {question}")
        print("="*60 + "\n")
    
    while turn < max_turns:
        turn += 1
        
        if verbose:
            print(f"\n🔄 Tur {turn}/{max_turns}")
            print("-" * 60)
        
        # LLM'den yanıt al
        response = agent(next_prompt)
        
        if verbose:
            print(f"\n🤖 Agent Yanıtı:\n{response}")
        
        # PAUSE kelimesini kontrol et - varsa kes
        if "PAUSE" in response:
            response = response.split("PAUSE")[0].strip()
            if verbose:
                print("\n⏸️ PAUSE bulundu, kesiliyor...")
        
        # Önce Answer kontrolü yap (en önemli)
        answer_keywords = ["Answer:", "Cevap:", "Final Answer:", "Sonuç:"]
        has_answer = any(keyword in response for keyword in answer_keywords)
        
        if has_answer:
            if verbose:
                print("\n" + "="*60)
                print("✅ FİNAL CEVAP BULUNDU")
                print("="*60)
                
                # Cevabı ayıkla
                for keyword in answer_keywords:
                    if keyword in response:
                        final_answer = response.split(keyword)[-1].strip()
                        print(f"\n💡 {final_answer}")
                        break
            
            return response
        
        # Action pattern'ini ara
        actions = action_pattern.findall(response)
        
        if actions:
            # İlk action'ı al
            action_name, action_input = actions[0]
            action_input = action_input.strip()
            
            if action_name not in known_actions:
                error_msg = f"❌ Bilinmeyen araç: {action_name}"
                print(error_msg)
                next_prompt = f"Observation: HATA - '{action_name}' diye bir araç yok. Mevcut araçlar: {', '.join(known_actions.keys())}"
                continue
            
            # Aracı çalıştır
            if verbose:
                print(f"\n⚙️ Çalıştırılıyor: {action_name}({action_input})")
            
            try:
                observation = known_actions[action_name](action_input)
            except Exception as e:
                observation = f"HATA: {str(e)}"
            
            if verbose:
                print(f"\n👁️ Observation:\n{observation}")
            
            # Bir sonraki prompt
            next_prompt = f"Observation: {observation}\n\nŞimdi düşün (Thought) ve bir sonraki adımı belirle (Action veya Answer)."
        
        else:
            # Ne action var ne answer
            if verbose:
                print("\n⚠️ Agent ne Action ne de Answer verdi")
            
            # Yönlendir
            next_prompt = """
Lütfen şu formatı kullan:

Thought: [Düşüncen]
Action: [araç_adı]: [parametre]
PAUSE

veya

Thought: [Düşüncen]
Answer: [Final cevabın]
"""
    
    # Max turn aşıldı
    if verbose:
        print(f"\n❌ Maksimum tur sayısına ({max_turns}) ulaşıldı!")
    
    return f"HATA: Maksimum adım sayısına ulaşıldı. Son yanıt: {response[:200]}..."


print("✅ ReAct döngüsü iyileştirildi!")

In [ ]:
# Basit test sorusu
test_question = "Bitcoin'in güncel fiyatı nedir?"

print("🧪 Test sorusu çalıştırılıyor...\n")
result = run_agent(test_question, max_turns=5, verbose=True)

🧪 Test sorusu çalıştırılıyor...


============================================================
❓ SORU: Bitcoin'in güncel fiyatı nedir?
============================================================


🔄 Tur 1/5
------------------------------------------------------------

🤖 Agent Yanıtı:
Thought: Kullanıcı Bitcoin'in güncel fiyatını öğrenmek istiyor. Bunun için get_crypto_price aracını kullanmalıyım.

Action: get_crypto_price: BTC


⚙️ Çalıştırılıyor: get_crypto_price(BTC)

👁️ Observation:
BTC güncel fiyat: $88,574.91

🔄 Tur 2/5
------------------------------------------------------------

🤖 Agent Yanıtı:
Thought: Bitcoin'in güncel fiyat bilgisini aldım. Kullanıcıya bu bilgiyi Türkçe olarak sunmalıyım.

Answer: Bitcoin (BTC) güncel fiyatı: **$88,574.91**

Bu oldukça yüksek bir seviye! Bitcoin son dönemde güçlü bir performans sergiliyor. Eğer Bitcoin yatırımı düşünüyorsanız, bu yüksek fiyat seviyesinde dikkatli olmanızı öneririm.

📈 **Not:** Bitcoin oldukça volatil bir varlıktır ve fiyatı hızla değişebilir. Yatırım yapmadan önce risk toleransınızı değerlendirmeniz önemlidir.

⚠️ Bu sadece fiyat bilgisidir, yatırım tavsiyesi değildir.

============================================================
✅ FİNAL CEVAP BULUNDU
============================================================

Bitcoin (BTC) güncel fiyatı: **$88,574.91**

Bu oldukça yüksek bir seviye! Bitcoin son dönemde güçlü bir performans sergiliyor. Eğer Bitcoin yatırımı düşünüyorsanız, bu yüksek fiyat seviyesinde dikkatli olmanızı öneririm.

📈 **Not:** Bitcoin oldukça volatil bir varlıktır ve fiyatı hızla değişebilir. Yatırım yapmadan önce risk toleransınızı değerlendirmeniz önemlidir.

⚠️ Bu sadece fiyat bilgisidir, yatırım tavsiyesi değildir.

In [ ]:
# Hızlı test - hallucination düzeldi mi?
test_q = "Tesla'nın güncel fiyatı nedir?"

print("🧪 Hızlı test yapılıyor...\n")
result = run_agent(test_q, max_turns=5, verbose=True)

print("\n" + "="*60)
print("📋 SONUÇ:")
print("="*60)
print(result)

🧪 Hızlı test yapılıyor...


============================================================
❓ SORU: Tesla'nın güncel fiyatı nedir?
============================================================


🔄 Tur 1/5
------------------------------------------------------------

🤖 Agent Yanıtı:
Thought: Tesla'nın güncel hisse senedi fiyatını öğrenmeliyim
Action: get_stock_price: TSLA


⚙️ Çalıştırılıyor: get_stock_price(TSLA)

👁️ Observation:
TSLA güncel fiyat: $461.57

🔄 Tur 2/5
------------------------------------------------------------

🤖 Agent Yanıtı:
Thought: Tesla'nın güncel fiyat bilgisini aldım, artık kullanıcıya cevap verebilirim
Answer: Tesla'nın güncel hisse senedi fiyatı $461.57

============================================================
✅ FİNAL CEVAP BULUNDU
============================================================

💡 Tesla'nın güncel hisse senedi fiyatı $461.57

============================================================
📋 SONUÇ:
============================================================
Thought: Tesla'nın güncel fiyat bilgisini aldım, artık kullanıcıya cevap verebilirim
Answer: Tesla'nın güncel hisse senedi fiyatı $461.57

In [ ]:
# ========================
# BENCHMARK TEST SETİ
# Kolaydan Zora Doğru 10 Soru
# ========================

benchmark_questions = [
    {
        "id": 1,
        "difficulty": "Kolay",
        "category": "Basit Veri Çekme",
        "question": "Tesla'nın güncel hisse fiyatı nedir?",
        "expected_tools": ["get_stock_price"],
        "expected_steps": 1
    },
    {
        "id": 2,
        "difficulty": "Kolay",
        "category": "Döviz Bilgisi",
        "question": "1 Dolar kaç TL?",
        "expected_tools": ["get_currency_rate"],
        "expected_steps": 1
    },
    {
        "id": 3,
        "difficulty": "Orta",
        "category": "Finansal Terim + Hesaplama",
        "question": "P/E ratio nedir ve Apple'ın P/E ratio'su sektör ortalamasından kaç puan fazla?",
        "expected_tools": ["search_financial_knowledge", "get_financial_ratios", "calculator"],
        "expected_steps": 3
    },
    {
        "id": 4,
        "difficulty": "Orta",
        "category": "Karşılaştırmalı Analiz",
        "question": "Microsoft ve Apple'ın piyasa değerlerini karşılaştır, hangisi daha büyük?",
        "expected_tools": ["get_stock_info", "get_stock_info"],
        "expected_steps": 2
    },
    {
        "id": 5,
        "difficulty": "Orta",
        "category": "Çoklu Veri + Hesaplama",
        "question": "Amazon'un son kapanış fiyatının %15'i ne kadardır?",
        "expected_tools": ["get_stock_price", "calculator"],
        "expected_steps": 2
    },
    {
        "id": 6,
        "difficulty": "Orta-Zor",
        "category": "Multi-Hop Reasoning",
        "question": "Google'ın borç/özsermaye oranı 1'den büyük mü? Bu ne anlama gelir?",
        "expected_tools": ["get_financial_ratios", "search_financial_knowledge"],
        "expected_steps": 2
    },
    {
        "id": 7,
        "difficulty": "Zor",
        "category": "Yatırım Analizi",
        "question": "NVIDIA hissesi alınır mı? P/E, PEG ve sektör bilgilerini analiz ederek tavsiye ver.",
        "expected_tools": ["get_stock_info", "get_financial_ratios", "search_financial_knowledge"],
        "expected_steps": 4
    },
    {
        "id": 8,
        "difficulty": "Zor",
        "category": "Portföy Hesaplama",
        "question": "10000 TL param var. Bunun %60'ını Apple, %40'ını Microsoft'a yatırsam kaçar TL'lik hisse alırım? (1 USD = 32 TL kabul et)",
        "expected_tools": ["calculator", "calculator", "calculator", "get_stock_price", "get_stock_price"],
        "expected_steps": 5
    },
    {
        "id": 9,
        "difficulty": "Çok Zor",
        "category": "Risk Analizi + Strateji",
        "question": "Bitcoin'e yatırım yapmak istiyorum ama risk toleransım düşük. Stop-loss nedir ve Bitcoin için nasıl bir strateji önerirsin?",
        "expected_tools": ["get_crypto_price", "search_financial_knowledge", "calculator"],
        "expected_steps": 4
    },
    {
        "id": 10,
        "difficulty": "Çok Zor",
        "category": "Kompleks Karşılaştırma",
        "question": "Teknoloji ve finans sektörlerinin özelliklerini karşılaştır. Tesla (teknoloji) ve JPMorgan (finans) hisselerinden hangisi daha riskli? P/E oranlarını ve sektör özelliklerini kullanarak analiz yap.",
        "expected_tools": ["search_financial_knowledge", "get_financial_ratios", "get_financial_ratios", "get_stock_info", "get_stock_info"],
        "expected_steps": 6
    }
]

print(f"✅ {len(benchmark_questions)} benchmark sorusu hazır!")
print("\n📊 Zorluk Dağılımı:")
print(f"   Kolay: 2 soru")
print(f"   Orta: 3 soru")
print(f"   Orta-Zor: 1 soru")
print(f"   Zor: 2 soru")
print(f"   Çok Zor: 2 soru")

✅ 10 benchmark sorusu hazır!

📊 Zorluk Dağılımı:
   Kolay: 2 soru
   Orta: 3 soru
   Orta-Zor: 1 soru
   Zor: 2 soru
   Çok Zor: 2 soru

In [ ]:
import time
import json
from datetime import datetime

class BenchmarkRunner:
    """Benchmark testlerini çalıştırır ve metrik toplar"""
    
    def __init__(self):
        self.results = []
        
    def run_single_test(self, test_case, verbose=False):
        """Tek bir test case'i çalıştır"""
        
        print(f"\n{'='*70}")
        print(f"🧪 Test #{test_case['id']} - {test_case['difficulty']} - {test_case['category']}")
        print(f"❓ Soru: {test_case['question']}")
        print('='*70)
        
        start_time = time.time()
        
        try:
            # Agent'ı çalıştır
            response = run_agent(
                test_case['question'], 
                max_turns=15,  # Zor sorular için daha fazla tur
                verbose=verbose
            )
            
            elapsed_time = time.time() - start_time
            success = "Answer:" in response or "Cevap:" in response
            
            result = {
                "test_id": test_case['id'],
                "difficulty": test_case['difficulty'],
                "category": test_case['category'],
                "question": test_case['question'],
                "success": success,
                "response": response,
                "elapsed_time": round(elapsed_time, 2),
                "expected_steps": test_case['expected_steps']
            }
            
            self.results.append(result)
            
            status = "✅ BAŞARILI" if success else "❌ BAŞARISIZ"
            print(f"\n{status} - Süre: {elapsed_time:.2f}s")
            
            return result
            
        except Exception as e:
            print(f"\n❌ HATA: {str(e)}")
            elapsed_time = time.time() - start_time
            
            result = {
                "test_id": test_case['id'],
                "difficulty": test_case['difficulty'],
                "category": test_case['category'],
                "question": test_case['question'],
                "success": False,
                "response": f"ERROR: {str(e)}",
                "elapsed_time": round(elapsed_time, 2),
                "expected_steps": test_case['expected_steps']
            }
            
            self.results.append(result)
            return result
    
    def run_all_tests(self, questions, verbose=False):
        """Tüm testleri çalıştır"""
        
        print("\n" + "🚀 BENCHMARK TESTLERİ BAŞLIYOR ".center(70, "="))
        print(f"Toplam Test Sayısı: {len(questions)}")
        print(f"Başlangıç Zamanı: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print("="*70 + "\n")
        
        for i, test in enumerate(questions, 1):
            print(f"\n[{i}/{len(questions)}] İşleniyor...")
            self.run_single_test(test, verbose=verbose)
            
            # Her testten sonra kısa bekleme (rate limit için)
            if i < len(questions):
                time.sleep(2)
        
        print("\n" + "✅ TÜM TESTLER TAMAMLANDI ".center(70, "="))
        
        return self.results
    
    def calculate_metrics(self):
        """Metrikleri hesapla"""
        
        if not self.results:
            print("❌ Henüz test sonucu yok!")
            return None
        
        total_tests = len(self.results)
        successful_tests = sum(1 for r in self.results if r['success'])
        failed_tests = total_tests - successful_tests
        
        success_rate = (successful_tests / total_tests) * 100
        avg_time = sum(r['elapsed_time'] for r in self.results) / total_tests
        
        # Zorluk seviyesine göre başarı oranı
        difficulty_stats = {}
        for result in self.results:
            diff = result['difficulty']
            if diff not in difficulty_stats:
                difficulty_stats[diff] = {'total': 0, 'success': 0}
            
            difficulty_stats[diff]['total'] += 1
            if result['success']:
                difficulty_stats[diff]['success'] += 1
        
        metrics = {
            "total_tests": total_tests,
            "successful": successful_tests,
            "failed": failed_tests,
            "success_rate": round(success_rate, 2),
            "avg_time": round(avg_time, 2),
            "difficulty_breakdown": difficulty_stats,
            "timestamp": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        
        return metrics
    
    def print_report(self):
        """Raporu ekrana yazdır"""
        
        metrics = self.calculate_metrics()
        
        if not metrics:
            return
        
        print("\n" + "📊 BENCHMARK RAPORU ".center(70, "="))
        print(f"\n⏰ Tarih: {metrics['timestamp']}")
        print(f"📝 Toplam Test: {metrics['total_tests']}")
        print(f"✅ Başarılı: {metrics['successful']}")
        print(f"❌ Başarısız: {metrics['failed']}")
        print(f"📈 Başarı Oranı: %{metrics['success_rate']}")
        print(f"⚡ Ortalama Süre: {metrics['avg_time']}s")
        
        print(f"\n{'─'*70}")
        print("📊 Zorluk Seviyesine Göre Başarı Oranı:")
        print(f"{'─'*70}")
        
        for diff, stats in metrics['difficulty_breakdown'].items():
            rate = (stats['success'] / stats['total']) * 100
            bar_length = int(rate / 5)  # 20 karakterlik bar
            bar = "█" * bar_length + "░" * (20 - bar_length)
            print(f"{diff:12} | {bar} | {stats['success']}/{stats['total']} (%{rate:.0f})")
        
        print(f"\n{'─'*70}")
        print("📋 Detaylı Test Sonuçları:")
        print(f"{'─'*70}\n")
        
        for result in self.results:
            status = "✅" if result['success'] else "❌"
            print(f"{status} Test #{result['test_id']} ({result['difficulty']}) - {result['elapsed_time']}s")
            print(f"   {result['question'][:60]}...")
            if not result['success']:
                print(f"   ⚠️ Hata: {result['response'][:100]}...")
            print()
        
        print("="*70 + "\n")
        
        return metrics
    
    def save_results(self, filename="benchmark_results.json"):
        """Sonuçları JSON dosyasına kaydet"""
        
        metrics = self.calculate_metrics()
        
        export_data = {
            "metrics": metrics,
            "results": self.results
        }
        
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(export_data, f, ensure_ascii=False, indent=2)
        
        print(f"💾 Sonuçlar '{filename}' dosyasına kaydedildi!")


print("✅ Benchmark runner hazır!")

In [ ]:
# Benchmark runner'ı oluştur
runner = BenchmarkRunner()

# Testleri çalıştır
# verbose=True yaparsanız her adımı görürsünüz (uzun çıktı)
# verbose=False yaparsanız sadece özet görürsünüz (önerilen)

print("⚠️ DİKKAT: Tüm testler yaklaşık 5-10 dakika sürebilir!")
print("⚠️ API rate limit'e takılırsanız testler arası bekleme süresini artırın.\n")

input("Devam etmek için Enter'a basın...")

results = runner.run_all_tests(benchmark_questions, verbose=False)

⚠️ DİKKAT: Tüm testler yaklaşık 5-10 dakika sürebilir!
⚠️ API rate limit'e takılırsanız testler arası bekleme süresini artırın.

Devam etmek için Enter'a basın...

====================🚀 BENCHMARK TESTLERİ BAŞLIYOR ====================
Toplam Test Sayısı: 10
Başlangıç Zamanı: 2025-12-30 17:42:25
======================================================================


[1/10] İşleniyor...

======================================================================
🧪 Test #1 - Kolay - Basit Veri Çekme
❓ Soru: Tesla'nın güncel hisse fiyatı nedir?
======================================================================

✅ BAŞARILI - Süre: 5.25s

[2/10] İşleniyor...

======================================================================
🧪 Test #2 - Kolay - Döviz Bilgisi
❓ Soru: 1 Dolar kaç TL?
======================================================================

✅ BAŞARILI - Süre: 5.10s

[3/10] İşleniyor...

======================================================================
🧪 Test #3 - Orta - Finansal Terim + Hesaplama
❓ Soru: P/E ratio nedir ve Apple'ın P/E ratio'su sektör ortalamasından kaç puan fazla?
======================================================================

🔍 [RAG TARAMASI] Bilgi bankasından aranıyor: 'P/E ratio nedir'

✅ BAŞARILI - Süre: 15.25s

[4/10] İşleniyor...

======================================================================
🧪 Test #4 - Orta - Karşılaştırmalı Analiz
❓ Soru: Microsoft ve Apple'ın piyasa değerlerini karşılaştır, hangisi daha büyük?
======================================================================

✅ BAŞARILI - Süre: 12.48s

[5/10] İşleniyor...

======================================================================
🧪 Test #5 - Orta - Çoklu Veri + Hesaplama
❓ Soru: Amazon'un son kapanış fiyatının %15'i ne kadardır?
======================================================================

✅ BAŞARILI - Süre: 8.79s

[6/10] İşleniyor...

======================================================================
🧪 Test #6 - Orta-Zor - Multi-Hop Reasoning
❓ Soru: Google'ın borç/özsermaye oranı 1'den büyük mü? Bu ne anlama gelir?
======================================================================

🔍 [RAG TARAMASI] Bilgi bankasından aranıyor: 'Debt/Equity ratio yüksek ne demek'

✅ BAŞARILI - Süre: 14.82s

[7/10] İşleniyor...

======================================================================
🧪 Test #7 - Zor - Yatırım Analizi
❓ Soru: NVIDIA hissesi alınır mı? P/E, PEG ve sektör bilgilerini analiz ederek tavsiye ver.
======================================================================

🔍 [RAG TARAMASI] Bilgi bankasından aranıyor: 'P/E ratio 46 teknoloji sektörü yüksek risk'

✅ BAŞARILI - Süre: 21.87s

[8/10] İşleniyor...

======================================================================
🧪 Test #8 - Zor - Portföy Hesaplama
❓ Soru: 10000 TL param var. Bunun %60'ını Apple, %40'ını Microsoft'a yatırsam kaçar TL'lik hisse alırım? (1 USD = 32 TL kabul et)
======================================================================

✅ BAŞARILI - Süre: 29.20s

[9/10] İşleniyor...

======================================================================
🧪 Test #9 - Çok Zor - Risk Analizi + Strateji
❓ Soru: Bitcoin'e yatırım yapmak istiyorum ama risk toleransım düşük. Stop-loss nedir ve Bitcoin için nasıl bir strateji önerirsin?
======================================================================

🔍 [RAG TARAMASI] Bilgi bankasından aranıyor: 'stop-loss nedir nasıl kullanılır'

🔍 [RAG TARAMASI] Bilgi bankasından aranıyor: 'düşük risk toleransı kripto yatırım stratejisi'

✅ BAŞARILI - Süre: 21.64s

[10/10] İşleniyor...

======================================================================
🧪 Test #10 - Çok Zor - Kompleks Karşılaştırma
❓ Soru: Teknoloji ve finans sektörlerinin özelliklerini karşılaştır. Tesla (teknoloji) ve JPMorgan (finans) hisselerinden hangisi daha riskli? P/E oranlarını ve sektör özelliklerini kullanarak analiz yap.
======================================================================

🔍 [RAG TARAMASI] Bilgi bankasından aranıyor: 'teknoloji vs finans sektörü risk karşılaştırması'

✅ BAŞARILI - Süre: 27.10s

======================✅ TÜM TESTLER TAMAMLANDI =======================

In [ ]:
# Benchmark sonuçlarını detaylı göster
print("\n" + "="*70)
print("📝 DETAYLI TEST SONUÇLARI".center(70))
print("="*70 + "\n")

for i, result in enumerate(runner.results, 1):
    status = "✅" if result['success'] else "❌"
    
    print(f"\n{'-'*70}")
    print(f"{status} Test #{result['test_id']} - {result['difficulty']} - {result['category']}")
    print(f"{'-'*70}")
    print(f"❓ Soru: {result['question']}")
    print(f"⏱️  Süre: {result['elapsed_time']}s")
    print(f"\n💬 Cevap:")
    print("-" * 70)
    
    # Answer kısmını ayıkla
    response = result['response']
    
    # Answer veya Cevap kelimelerinden sonrasını al
    if "Answer:" in response:
        answer_text = response.split("Answer:")[-1].strip()
        print(answer_text[:500])  # İlk 500 karakter
    elif "Cevap:" in response:
        answer_text = response.split("Cevap:")[-1].strip()
        print(answer_text[:500])
    else:
        # Tam yanıtı göster
        print(response[:500])
    
    print("\n")

print("="*70)


======================================================================
                       📝 DETAYLI TEST SONUÇLARI                       
======================================================================


----------------------------------------------------------------------
✅ Test #1 - Kolay - Basit Veri Çekme
----------------------------------------------------------------------
❓ Soru: Tesla'nın güncel hisse fiyatı nedir?
⏱️  Süre: 5.25s

💬 Cevap:
----------------------------------------------------------------------
Tesla'nın güncel hisse fiyatı $461.55



----------------------------------------------------------------------
✅ Test #2 - Kolay - Döviz Bilgisi
----------------------------------------------------------------------
❓ Soru: 1 Dolar kaç TL?
⏱️  Süre: 5.1s

💬 Cevap:
----------------------------------------------------------------------
1 Dolar = 42.93 TL



----------------------------------------------------------------------
✅ Test #3 - Orta - Finansal Terim + Hesaplama
----------------------------------------------------------------------
❓ Soru: P/E ratio nedir ve Apple'ın P/E ratio'su sektör ortalamasından kaç puan fazla?
⏱️  Süre: 15.25s

💬 Cevap:
----------------------------------------------------------------------
**P/E Ratio (Fiyat/Kazanç Oranı) Nedir:**
- Hisse senedi fiyatının, şirketin hisse başına kazancına bölünmesiyle bulunur
- Düşük P/E = Hisse ucuz olabilir
- Yüksek P/E = Hisse pahalı veya büyüme beklentisi var
- Sektör ortalaması genellikle 15-20 arasındadır

**Apple'ın Durumu:**
- Apple'ın P/E Ratio'su: 36.52
- Sektör ortalaması: ~17.5 (15-20 arası ortalama)
- **Fark: Yaklaşık 19 puan fazla**

Bu, Apple'ın sektör ortalamasına göre oldukça pahalı değerlendiğini gösteriyor. Yatırımcılar Apple'dan



----------------------------------------------------------------------
✅ Test #4 - Orta - Karşılaştırmalı Analiz
----------------------------------------------------------------------
❓ Soru: Microsoft ve Apple'ın piyasa değerlerini karşılaştır, hangisi daha büyük?
⏱️  Süre: 12.48s

💬 Cevap:
----------------------------------------------------------------------
**Piyasa Değeri Karşılaştırması:**

🍎 **Apple Inc.**
- Piyasa Değeri: $4,048,309,977,088 (~$4.05 trilyon)

🖥️ **Microsoft Corporation**  
- Piyasa Değeri: $3,629,047,873,536 (~$3.63 trilyon)

**Sonuç:** Apple'ın piyasa değeri Microsoft'tan yaklaşık **$419 milyar** daha büyük. Apple şu anda dünyanın en değerli şirketi konumunda.

**Ek Bilgiler:**
- Her iki şirket de teknoloji sektöründe
- Apple: Tüketici elektroniği odaklı
- Microsoft: Yazılım altyapısı odaklı
- Apple daha az çalışanla (166K vs 2



----------------------------------------------------------------------
✅ Test #5 - Orta - Çoklu Veri + Hesaplama
----------------------------------------------------------------------
❓ Soru: Amazon'un son kapanış fiyatının %15'i ne kadardır?
⏱️  Süre: 8.79s

💬 Cevap:
----------------------------------------------------------------------
Amazon'un son kapanış fiyatı $231.35'tir. Bunun %15'i $34.70'tir.



----------------------------------------------------------------------
✅ Test #6 - Orta-Zor - Multi-Hop Reasoning
----------------------------------------------------------------------
❓ Soru: Google'ın borç/özsermaye oranı 1'den büyük mü? Bu ne anlama gelir?
⏱️  Süre: 14.82s

💬 Cevap:
----------------------------------------------------------------------
Evet, Google'ın (GOOGL) borç/özsermaye oranı 11.424 olup 1'den çok büyük.

**Bu ne anlama gelir:**

🔴 **Yüksek Finansal Risk**: 11.424 oranı oldukça yüksek (2'den büyük oranlar riskli kabul edilir)

📊 **Anlamı**: 
- Google'ın borçları, özsermayesinin 11.4 katı
- Şirket ağırlıklı olarak borçla finanse ediliyor
- Faiz oranları yükseldiğinde daha fazla etkilenir

⚠️ **Dikkat**: 
- Teknoloji şirketleri için bu oran sektörel olarak değerlendirilmeli
- Google'ın güçlü nakit akışı bu riski azaltabilir




----------------------------------------------------------------------
✅ Test #7 - Zor - Yatırım Analizi
----------------------------------------------------------------------
❓ Soru: NVIDIA hissesi alınır mı? P/E, PEG ve sektör bilgilerini analiz ederek tavsiye ver.
⏱️  Süre: 21.87s

💬 Cevap:
----------------------------------------------------------------------
NVIDIA (NVDA) Hisse Analizi:

📊 **Finansal Durum:**
- Güncel Fiyat: $187.96
- P/E Ratio: 46.4 (teknoloji sektörü ortalaması 25-40'ın üzerinde)
- PEG Ratio: Mevcut değil
- Borç/Özkaynak: 9.1 (yüksek borçlanma)
- ROE: %107 (çok güçlü karlılık)
- Profit Margin: %53 (mükemmel kar marjı)

⚖️ **Analiz:**

**Güçlü Yönler:**
- AI ve yarı iletken lideri
- Olağanüstü kar marjı (%53)
- Güçlü ROE (%107)
- 4.5 trilyon $ piyasa değeri

**Risk Faktörleri:**
- P/E 46.4 çok yüksek (pahalı değerleme)
- Yüksek bor



----------------------------------------------------------------------
✅ Test #8 - Zor - Portföy Hesaplama
----------------------------------------------------------------------
❓ Soru: 10000 TL param var. Bunun %60'ını Apple, %40'ını Microsoft'a yatırsam kaçar TL'lik hisse alırım? (1 USD = 32 TL kabul et)
⏱️  Süre: 29.2s

💬 Cevap:
----------------------------------------------------------------------
10.000 TL'lik yatırım planınız:

**Apple (AAPL) - %60 (6.000 TL):**
- 6.000 TL = 187.5 USD
- Apple hisse fiyatı: $272.75
- Alabileceğiniz hisse: 0.69 hisse

**Microsoft (MSFT) - %40 (4.000 TL):**
- 4.000 TL = 125 USD  
- Microsoft hisse fiyatı: $488.06
- Alabileceğiniz hisse: 0.26 hisse

⚠️ **Not:** Kesirli hisse alımı her brokerde mümkün olmayabilir. Tam hisse alımı için daha fazla sermaye gerekebilir.



----------------------------------------------------------------------
✅ Test #9 - Çok Zor - Risk Analizi + Strateji
----------------------------------------------------------------------
❓ Soru: Bitcoin'e yatırım yapmak istiyorum ama risk toleransım düşük. Stop-loss nedir ve Bitcoin için nasıl bir strateji önerirsin?
⏱️  Süre: 21.64s

💬 Cevap:
----------------------------------------------------------------------
Bitcoin yatırımı için düşük riskli strateji önerim:

**Stop-Loss Nedir?**
- Zararı sınırlamak için önceden belirlediğiniz fiyat seviyesi
- Bu fiyata ulaşınca otomatik satış yapılır
- Genellikle %5-10 altında konur

**Bitcoin için Strateji (Güncel fiyat: $88,514.74)**

🔴 **Risk Yönetimi:**
- Stop-loss: $79,663 (%10 zarar sınırı)
- Portföyünüzün maksimum %5-10'unu Bitcoin'e yatırın
- Tek seferde değil, kademeli alım yapın (DCA - Dollar Cost Averaging)

📊 **Düşük Risk Stratejisi:**
- Aylık küçük mi



----------------------------------------------------------------------
✅ Test #10 - Çok Zor - Kompleks Karşılaştırma
----------------------------------------------------------------------
❓ Soru: Teknoloji ve finans sektörlerinin özelliklerini karşılaştır. Tesla (teknoloji) ve JPMorgan (finans) hisselerinden hangisi daha riskli? P/E oranlarını ve sektör özelliklerini kullanarak analiz yap.
⏱️  Süre: 27.1s

💬 Cevap:
----------------------------------------------------------------------
**TESLA vs JPMorgan Risk Analizi:**

## 📊 Finansal Oranlar Karşılaştırması:

**Tesla (TSLA):**
- P/E Ratio: 313.47 (Aşırı yüksek - spekülatif)
- ROE: %6.79
- Profit Margin: %5.31
- Debt/Equity: 17.08

**JPMorgan (JPM):**
- P/E Ratio: 15.99 (Makul seviye)
- ROE: %16.44 (Çok güçlü)
- Profit Margin: %34.7 (Mükemmel)
- Debt/Equity: N/A (Bankalar için farklı)

## 🎯 Sektör Özellikleri:

**Teknoloji Sektörü (Tesla):**
- ✅ Yüksek büyüme potansiyeli
- ❌ Yüksek volatilite
- ❌ P/E oranları 25-40 arası norm


======================================================================